In [ ]:
import json
from pathlib import Path
import langextract as lx
import textwrap
import os

In [26]:
# Load data (same as previous notebooks)
data_file = Path("../data/output_03e48481195ba4783678f1ae446b40a7f6f12791.jsonl")

def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        return json.loads(file.readline())

# Load and get text section
data = read_jsonl(data_file)
full_text = data['text']

# Get chapter 2 section (same as other notebooks)
pagelookup = {page[-1]: page[0] for page in data['attributes']['pdf_page_numbers']}
text_section = full_text[pagelookup[33]:pagelookup[34]-1]

print(f"Processing text section of {len(text_section):,} characters")

Processing text section of 2,225 characters


In [27]:
print(text_section)

2 | THE PERSISTENCE OF THE WORD

(There Is No Dictionary in the Mind)

Odysseus wept when he heard the poet sing of his great deeds abroad because, once sung, they were no longer his alone. They belonged to anyone who heard the song.

—Ward Just (2004)

“TRY TO IMAGINE,” proposed Walter J. Ong, Jesuit priest, philosopher, and cultural historian, “a culture where no one has ever ‘looked up’ anything.” To subtract the technologies of information internalized over two millennia requires a leap of imagination backward into a forgotten past. The hardest technology to erase from our minds is the first of all: writing. This arises at the very dawn of history, as it must, because the history begins with the writing. The pastness of the past depends on it.

It takes a few thousand years for this mapping of language onto a system of signs to become second nature, and then there is no return to naïveté. Forgotten is the time when our very awareness of words came from seeing them. “In a primary or

In [28]:
# Define comprehensive prompt and examples for complex literary text
prompt = textwrap.dedent("""\
    Extract characters, topics, and relationships from the given text.

    Provide meaningful attributes for every entity to add context and depth.

    Important: Use exact text from the input for extraction_text. Do not paraphrase.
    Extract entities in order of appearance with no overlapping text spans.

    Note: In book, names appear under different forms.""")

In [29]:
examples = [
    lx.data.ExampleData(
        text=textwrap.dedent("""\
            The deepest consequences of writing, for the individual and for the culture, 
            could hardly have been foreseen, but even Plato could see some of the 
            power of this disconnection. The one speaks to the multitude. 
            The dead speak to the living, the living to the unborn. As McLuhan said,
            “Two thousand years of manuscript culture lay ahead of the Western world 
            when Plato made this observation.”"""),
        extractions=[
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="Plato",
                attributes={"role": "philosopher"}
            ),
            lx.data.Extraction(
                extraction_class="topic",
                extraction_text="The consequences of writing",
                attributes={"character": "Plato"}
            ),
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="McLuhan",
                attributes={"role": "commenter"}
            ),
            lx.data.Extraction(
                extraction_class="relationship",
                extraction_text="As McLuhan said, “Two thousand years of manuscript culture lay ahead of the Western world when Plato made this observation",
                attributes={"type": "support", "character_1": "McLuhan", "character_2": "Platop"}
            ),
        ]
    )
]

In [33]:
result = lx.extract(
    text_or_documents=text_section,
    prompt_description=prompt,
    examples=examples,
    language_model_type=lx.inference.OllamaLanguageModel,
    model_id="llama3.2",  
    model_url=os.getenv("OLLAMA_HOST", "http://localhost:11434"),
    temperature=0.3,
    fence_output=False,
    use_schema_constraints=False
)

LangExtract: Processing, current=2,223 chars, processed=2,223 chars:  [00:30]


ValueError: Ollama Model timed out (timeout=30, num_threads=None)

In [25]:
?lx.extract

Signature:
lx.extract(
    text_or_documents: 'str | data.Document | Iterable[data.Document]',
    prompt_description: 'str | None' = None,
    examples: 'Sequence[data.ExampleData] | None' = None,
    model_id: 'str' = 'gemini-2.5-flash',
    api_key: 'str | None' = None,
    language_model_type: 'Type[LanguageModelT]' = <class 'langextract.inference.GeminiLanguageModel'>,
    format_type: 'data.FormatType' = <FormatType.JSON: 'json'>,
    max_char_buffer: 'int' = 1000,
    temperature: 'float' = 0.5,
    fence_output: 'bool' = False,
    use_schema_constraints: 'bool' = True,
    batch_length: 'int' = 10,
    max_workers: 'int' = 10,
    additional_context: 'str | None' = None,
    resolver_params: 'dict | None' = None,
    language_model_params: 'dict | None' = None,
    debug: 'bool' = True,
    model_url: 'str | None' = None,
    extraction_passes: 'int' = 1,
) -> 'data.AnnotatedDocument | Iterable[data.AnnotatedDocument]'
Docstring:
Extracts structured information from text.

Ret

In [ ]:
print(f"Extracted {len(result.extractions)} entities from {len(result.text):,} characters")